In [13]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [8]:
df = pd.read_csv('../data/raw/CriticalPath_Data_EM_Confidential.csv')

Make `df['enrolled']` a T/F categorical value to eliminate NaNs

In [9]:
df['Enrolled'] = df['Enrolled'].map({80:True})
df['Enrolled'] = df['Enrolled'].fillna(False)

Eliminate all HEOP and Albany Med applicants from `df['Application_Type']` in order to lessen noise

In [12]:
df = df[(df['Application_Type']!='AM') & (df['Application_Type']!='HE')]

Write this new data set to a new file.

In [12]:
df.to_csv('../data/processed/CriticalPath_Data_EM_Confidential_lessNoise.csv')